# Data Processing and Model Training Notebook

In [7]:
!{os.sys.executable} -m pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 4.2 MB/s eta 0:00:0000:0100:01m


In [8]:
# import packages
import os
import cv2
import numpy as np
from IPython.display import Video, display
import time


from utils.pretrained_deployment import download_images, download_images2, download_images_with_resize, download_images_full_size
from utils.display import *
from utils.make_dataset_nyc_landmarks import make_nyc_dataset

from bing_image_downloader import downloader

device = 'cuda'

# reload modules every 2 seconds to see changes in notebook
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Download the Images

In [9]:
#########################################################
# ONLY RUN IF YOU WANT TO REDOWNLOAD IMAGES
#########################################################


def download_images(query, num_images, download_path):
    downloader.download(query, limit=num_images, output_dir=download_path, adult_filter_off=True, force_replace=False, verbose = False)

queries = ["empire_state_building_from_the_ground",
"empire_state_building",
"empire_state_building_nyc",
"1_world_trade_center_from_the_ground",
"1_world_trade_center",
"1_world_trade_center_nyc", 
"432_park_ave_from_the_ground", 
"432_park_ave",
"432_park_ave_nyc", 
"united_nations_building_river_view",
"united_nations_building",
"united_nations_building_nyc",
"flatiron_side_view",
"flatiron",
"flatiron_nyc",
"brooklyn_bridge",
"brooklyn_bridge_nyc",
"brooklyn_bridge_image",
"chrysler_building_from_the_ground",
"chrysler_building",
"chrysler_building_nyc"
"metlife_building_nyc",
"metlife_building_front",
"metlife_building",
"statue_of_liberty",
"statue_of_liberty_nyc",
"statue_of_liberty_new_york",
"30_hudson_yards_from_the_ground",
"30_hudson_yards",
"30_hudson_yards_nyc",
"NYC_skyline",
"NYC_landmarks",
"NYC_view_from_NJ"
          ]

classes = {
    0: "EmpireState",
    1: "WTC",
    2: "432ParkAve",
    3: "UNBuilding",
    4: "Flatiron",
    5: "BrooklynBridge",
    6: "ChryslerBuilding",
    7: "MetlifeBuilding",
    8: "StatueOfLiberty",
    9: "30HudsonYards",
}

#Make sure the yaml file matches this ^^



# query = 'empire_state_building_from_the_ground'
num_images = 100  # Number of images to download
download_path = "./downloaded_images"  # Target folder path
for query in queries:

    # download_images(query, num_images, download_path) #uncomment if you want new data
    time.sleep(0.5)
        

# Add annotations using rectlabel

Use the available software to create the yolo annotations for each image. This will take some time so go ahead, put some podcasts and get cracking. The .txt files need to be saved in the same directory as the original images. Make sure that the labels are following the same classes dictionary that we have here or things will not go well. 

# Place into valid and training folders, prepare yolo datasets

In [10]:
#########################################################
# ONLY RUN IF YOU WANT DELETE CURRENT DATA AND REMAKE SET
#########################################################

fraction_valid = 0.25
train_path = "./datasets/nyc_landmarks/train"
valid_path = "./datasets/nyc_landmarks/valid"

make_nyc_dataset(
    download_path,
    train_path = train_path,
    val_path = valid_path,
    label_dictionary = classes,
    fraction_valid = fraction_valid
)

Processing folder: metlife_building
Raw Name: Image_9
Combo Name: metlife_building_Image_9
Skipping copying of metlife_building_Image_9 as corresponding .jpg does not exist.
Raw Name: Image_8
Combo Name: metlife_building_Image_8
Raw Name: Image_28
Combo Name: metlife_building_Image_28
Raw Name: Image_15
Combo Name: metlife_building_Image_15
Raw Name: Image_29
Combo Name: metlife_building_Image_29
Raw Name: Image_17
Combo Name: metlife_building_Image_17
Raw Name: Image_16
Combo Name: metlife_building_Image_16
Raw Name: Image_12
Combo Name: metlife_building_Image_12
Raw Name: Image_13
Combo Name: metlife_building_Image_13
Raw Name: Image_11
Combo Name: metlife_building_Image_11
Raw Name: Image_39
Combo Name: metlife_building_Image_39
Raw Name: Image_10
Combo Name: metlife_building_Image_10
Raw Name: Image_21
Combo Name: metlife_building_Image_21
Raw Name: Image_34
Combo Name: metlife_building_Image_34
Raw Name: Image_36
Combo Name: metlife_building_Image_36
Raw Name: Image_22
Combo Name:

## Verify Dataset

# Train Yolov9

In [11]:
from ultralytics import YOLO

# Build a YOLOv9c model from pretrained weight
model = YOLO('yolov9c.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='dataset.yaml', epochs=1, imgsz=640)

YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs
New https://pypi.org/project/ultralytics/8.2.0 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.29 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=dataset.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/train... 944 images, 0 backgrounds, 0 corrupt: 100%|██████████| 944/944 [00:01<00:00, 861.03it/s]

train: WARNING ⚠️ /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/train/1_world_trade_center_nyc_Image_16.jpg: corrupt JPEG restored and saved
train: New cache created: /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/train.cache



val: Scanning /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 3334.00it/s]

val: New cache created: /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid.cache


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.415      5.168      1.629         49        640:   8%|▊         | 5/59 [08:51<1:35:44, 106.38s/it]


KeyboardInterrupt: 

# Evaluation

In [44]:
from PIL import Image
import matplotlib.pyplot as plt

# Predict bounding boxes on validation data
results = model.predict(valid_path)


# for result in results:
#     print(result)




image 1/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/1_world_trade_center_Image_3.jpg: 640x448 (no detections), 340.7ms
image 2/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/432_park_ave_Image_3.jpg: 640x448 (no detections), 329.4ms
image 3/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/empire_state_building_from_the_ground_Image_3.jpg: 640x448 (no detections), 329.0ms
image 4/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/flatiron_Image_3.jpg: 640x448 (no detections), 314.0ms
image 5/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/united_nations_building_Image_3.jpg: 480x640 (no 

# Train RT-DETR

In [6]:
from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='dataset.yaml', epochs=1, imgsz=640)

ModuleNotFoundError: No module named 'pandas'

# Archive

In [ ]:
import time
from google_images_download import google_images_download

def download_images(query, num_images, download_path):
    # Create an instance of the google_images_download class
    image_downloader = google_images_download.googleimagesdownload()

    # Define the arguments for the download
    arguments = {
        "keywords": query,
        "limit": num_images,
        "output_directory": download_path,
        "no_directory": False,  # Save images directly in the output directory without creating subfolders
        "no_download_if_exists": False,  # Download images even if they already exist
        "silent_mode": False,  # Suppress output messages
    }

    # Download the images
    image_downloader.download(arguments)

queries = ["empire_state_building_from_the_ground", 
"1_world_trade_center", 
"432_park_ave", 
"united_nations_building",
"flatiron",
"brooklyn_bridge",
"chrysler_building",
"metlife_building",
"state_of_liberty",
"30_hudson_yards"]

classes = {
    0: "EmpireState",
    1: "WTC",
    2: "432ParkAve",
    3: "UNBuilding",
    4: "Flatiron",
    5: "BrooklynBridge",
    6: "ChryslerBuilding",
    7: "MetlifeBuilding",
    8: "StatueOfLiberty",
    9: "30HudsonYards",
}

# Number of images to download (set to 0 for unlimited)
num_images = 1
download_path = "./downloaded_images"  # Target folder path

for query in queries:
    download_images(query, num_images, download_path)
    time.sleep(0.5)  # Sleep for a short duration to avoid overwhelming the server
